In [18]:
#importamos las librerías necesarias para trabajar
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import pandas as pd
import ast
import matplotlib.pyplot as plt
from textblob import TextBlob

In [19]:
#cargamos el archivo json a una lista vacía
data = []
with open('C:\\Users\\Tom\\Desktop\\PIMLOps-STEAM\\dataset\\output_steam_games.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

#creamos el dataframe
df_steam_games = pd.DataFrame(data)


In [20]:
#eliminamos los datos nulos y duplicados de la columna "app_name"
df_steam_games.dropna(subset="app_name", inplace=True)
df_steam_games.drop_duplicates(subset="app_name", inplace=True)


In [21]:
#creamos la columna "Release Date" que llenaremos con datos extraidos de la columna "release_date"
df_steam_games['Release Date'] = df_steam_games['release_date'].str.extract(r'(\d{4})')
#eliminamos la columna que ya no utilizamos
df_steam_games.drop('release_date' , axis = 1, inplace = True)

In [22]:
#convertimos los datos nulos de la columna "genres" en cadenas vacías
df_steam_games['genres'].fillna('', inplace=True)
#creamos la columna que obtienes valores booleanos "Is Free To Play" que la llenamos con los resultados de aplicar lambda a los valores de la columna "genres"
df_steam_games['Is Free To Play'] = df_steam_games['genres'].apply(lambda x: 'Free to Play' in str(x))
#esta linea selecciona las filas donde "Is Free To Play" es True y donde el "Price" es diferente de 0, y establece el "Price" de esas filas en 0.0
df_steam_games.loc[df_steam_games['Is Free To Play'] & df_steam_games['price'], 'price'] = 0.0

In [23]:
#aquí renombramos las columnas para que sea vean más prolijas y esteticas
df_steam_games.rename(columns={'publisher': 'Publisher'}, inplace=True)
df_steam_games.rename(columns={'genres': 'Genres'}, inplace=True)
df_steam_games.rename(columns={'app_name': 'App Name'}, inplace=True)
df_steam_games.rename(columns={'url': 'URL'}, inplace=True)
df_steam_games.rename(columns={'tags': 'Tags'}, inplace=True)
df_steam_games.rename(columns={'reviews_url': 'Reviews URL'}, inplace=True)
df_steam_games.rename(columns={'specs': 'Specs'}, inplace=True)
df_steam_games.rename(columns={'price': 'Price'}, inplace=True)
df_steam_games.rename(columns={'early_access': 'Early Access'}, inplace=True)
df_steam_games.rename(columns={'id': 'ID'}, inplace=True)
df_steam_games.rename(columns={'developer': 'Developer'}, inplace=True)

In [24]:
#eliminamos la columna "title" ya que contiene los mismos datos que la columna "App Name" y además contenía datos nulos
df_steam_games.drop(columns="title", inplace=True)

In [25]:
#cambiamos los datos nulos por cadenas vacias, a excepción de la columna "Price" la cual contendrá como valor 0.0
df_steam_games['Tags'].fillna('', inplace=True)
df_steam_games['Specs'].fillna('', inplace=True)
df_steam_games['Price'].fillna(0.0, inplace=True)
df_steam_games['Developer'].fillna('', inplace=True)
df_steam_games['Genres'].fillna('', inplace=True)
#eliminamos nulos de las columnas "Release Date" y "Publisher"
df_steam_games.dropna(subset="Release Date", inplace=True)
df_steam_games.dropna(subset="Publisher", inplace=True)

In [26]:
#Reiniciamos el indice
df_steam_games=df_steam_games.reset_index(drop=True)

In [27]:
#estas lineas aplican la funcion lambda a las columnas. Verifica cada valor en las columnas, y si un valor no es una lista, lo colca dentro de una.
#Si un valor ya es una lista, lo deja tal como está.
df_steam_games['Genres'] = df_steam_games['Genres'].apply(lambda x: [x] if not isinstance(x, list) else x)
df_steam_games['Tags'] = df_steam_games['Tags'].apply(lambda x: [x] if not isinstance(x, list) else x)
df_steam_games['Specs'] = df_steam_games['Specs'].apply(lambda x: [x] if not isinstance(x, list) else x)

In [28]:
#Esta linea de codigo está reemplazando todos los valores de cadena en la columna "Price" con 0.0
df_steam_games['Price'] = df_steam_games['Price'].apply(lambda x: 0.0 if isinstance(x, str) else x)

In [29]:
#Esta linea de codigo está reemplazando "Free_To_Play" con 0.0 en la columna "Price" y luego convierte todos los valores a numeros flotantes
df_steam_games['Price'] = df_steam_games['Price'].replace('Free_To_Play', 0.0).astype(float)

In [30]:
#Convertimos los valores de la columna "Release Date" en numeros enteros
df_steam_games['Release Date'] = df_steam_games['Release Date'].astype(int)

In [31]:
#Verificamos si hay algún valor en la columna "Release Date" que sea una cadena(str)
any(df_steam_games['Release Date'].apply(lambda x: isinstance(x, str)))

False

In [32]:
#Llamamos al dataframe para ver como quedó
df_steam_games

,Publisher,Genres,App Name,URL,Tags,Reviews URL,Specs,Price,Early Access,ID,Developer,Release Date,Is Free To Play
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018,False
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,2018,True
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,2017,True
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017,False
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,2018,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23934,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games,2018,False
23935,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018,False
23936,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018,False
23937,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018,False


In [33]:
column_data_types = df_steam_games['Genres'].dtypes
print(column_data_types)

object


In [37]:
# Convertir la columna "Genres" a string
df_steam_games["Genres"] = df_steam_games["Genres"].astype(str)

# Ahora puedes continuar con el resto de tu código

In [38]:
column_data_types = df_steam_games['Genres'].dtypes
print(column_data_types)

object


In [35]:
#creamos un archivo parquet del dataframe
df_steam_games.to_parquet('Steam_Games.parquet')